In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [20]:
df = pd.read_csv('k8s_pod_metrics.csv')

In [21]:
string_cols = ['namespace', 'pod', 'node','deployment']
label_encoders = {}

for col in string_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [22]:
target_columns = ['deployment Replica Mismatch', 'deployment Unavailable Pods', 'deployment ImagePullFailure', 'deployment CrashLoopBackOff', 'deployment FailedScheduling', 'deployment QuotaExceeded', 'deployment ProgressDeadlineExceeded','CPU Pressure', 'Memory Pressure', 'Disk Pressure', 'Network Unavailable', 'Node Not Ready', 'PID Pressure', 'Node Unschedulable','CPU Throttling', 'High CPU Usage', 'OOMKilled (Out of Memory)','CrashLoopBackOff', 'ContainerNotReady', 'PodUnschedulable', 'NodePressure', 'ImagePullFailure']
targets = df[target_columns]
features = df.drop(columns=target_columns)
features.fillna(0, inplace=True)
targets.fillna(0, inplace=True)


C:\Users\adith\AppData\Local\Temp\ipykernel_3712\2464279806.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets.fillna(0, inplace=True)


In [23]:
features['timestamp'] = pd.to_datetime(features['timestamp']).astype('int64')
scaler_features = MinMaxScaler()
features_scaled = scaler_features.fit_transform(features)

scaler_targets = MinMaxScaler()
targets_scaled = scaler_targets.fit_transform(targets)

In [24]:
def create_sequences(X, y, time_steps=10):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:i+time_steps])
        y_seq.append(y[i+time_steps])
    return np.array(X_seq), np.array(y_seq)

time_steps = 10
X_seq, y_seq = create_sequences(features_scaled, targets_scaled, time_steps)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

In [26]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dense(22))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()

c:\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 128)            │       109,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 22)             │         1,430 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,254 (465.84 KB)

 Trainable params: 119,254 (465.84 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
loaded_model = tf.keras.models.load_model("lstm_model.h5", compile=False)

loaded_model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

loaded_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 128)            │       109,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 22)             │         1,430 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,254 (465.84 KB)

 Trainable params: 119,254 (465.84 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

Epoch 1/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0312 - mae: 0.0996 - val_loss: 0.0093 - val_mae: 0.0313
Epoch 2/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0093 - mae: 0.0330 - val_loss: 0.0083 - val_mae: 0.0298
Epoch 3/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0089 - mae: 0.0310 - val_loss: 0.0082 - val_mae: 0.0286
Epoch 4/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0084 - mae: 0.0315 - val_loss: 0.0072 - val_mae: 0.0266
Epoch 5/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0079 - mae: 0.0289 - val_loss: 0.0070 - val_mae: 0.0253
Epoch 6/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0073 - mae: 0.0296 - val_loss: 0.0066 - val_mae: 0.0268
Epoch 7/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0066 - mae: 0.0269 - val_loss: 0.0059 - val_mae: 0.0241
Epoch 8/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0063 - mae: 0.0267 - val_loss: 0.0057 - val_mae: 0.0261
Epoch 9/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0060 - mae: 

In [29]:
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0058 - mae: 0.0198 
Test Loss: 0.006931221578270197, Test MAE: 0.021620724350214005


In [30]:
predictions = model.predict(X_test)
print(predictions.shape)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
(518, 22)
